<a href="https://colab.research.google.com/github/s4mto/ML_Apache_Pipeline_Final_Project/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Final Project - Build an ML Pipeline for Airfoil noise prediction


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at an aeronautics consulting company. Your company prides itself in being able to efficiently design airfoils for use in planes and sports cars. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able to do ETL jobs and build ML pipelines. In this project you will use the modified version of the NASA Airfoil Self Noise dataset. You will clean this dataset, by dropping the duplicate rows, and removing the rows with null values. You will create an ML pipe line to create a model that will predict the SoundLevel based on all the other columns. You will evaluate the model and towards the end you will persist the model.



## Objectives

In this 4 part assignment you will:

- Part 1 Perform ETL activity
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Create a  Machine Learning Pipeline
  - Create a machine learning pipeline for prediction
- Part 3 Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 Persist the Model
  - Save the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - The original dataset can be found here NASA airfoil self noise dataset. https://archive.ics.uci.edu/dataset/291/airfoil+self+noise

 - This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.


Diagram of an airfoil. - For informational purpose


![Airfoil with flow](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_with_flow.png)


Diagram showing the Angle of attack. - For informational purpose


![Airfoil angle of attack](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_angle_of_attack.jpg)


## Before you Start


Before you start attempting this project it is highly recommended that you finish the practice project.


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
 connect to this cluster.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity


### Task 1 - Import required libraries


In [ ]:
#your code goes here
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer

from pyspark.sql import SparkSession


# import functions/Classes for pipeline creation

from pyspark.ml import Pipeline

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


### Task 2 - Create a spark session


In [ ]:
#Create a SparkSession

spark = SparkSession.builder.appName("Airfoil Noise Prediction").getOrCreate()

23/09/13 12:46:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Task 3 - Load the csv file into a dataframe


Download the data file.

NOTE : Please ensure you use the dataset below and not the original dataset mentioned above.


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


--2023-09-13 12:47:22--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.001s  

2023-09-13 12:47:22 (40.0 MB/s) - ‘NASA_airfoil_noise_raw.csv’ saved [60682/60682]



Load the dataset into the spark dataframe


In [ ]:
# Load the dataset that you have downloaded in the previous task

df = spark.read.csv("NASA_airfoil_noise_raw.csv", header=True, inferSchema=True)


### Task 4 - Print top 5 rows of the dataset


In [ ]:
#your code goes here
df.show(5)

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



### Task 6 - Print the total number of rows in the dataset


In [ ]:
#your code goes here
rowcount1 = df.count()
print(rowcount1)

1522


### Task 7 - Drop all the duplicate rows from the dataset


In [ ]:
df = df.dropDuplicates()

### Task 8 - Print the total number of rows in the dataset


In [ ]:
#your code goes here

rowcount2 = df.count()
print(rowcount2)


[Stage 17:===================================================>  (190 + 8) / 200]

1503


### Task 9 - Drop all the rows that contain null values from the dataset


In [ ]:
df=df.dropna()

### Task 10 - Print the total number of rows in the dataset


In [ ]:
#your code goes here

rowcount3 = df.count()
print(rowcount3)


[Stage 20:=====================================================>(199 + 1) / 200]

1499


### Task 11 - Rename the column "SoundLevel" to "SoundLevelDecibels"Drop


In [ ]:
# your code goes here

df.printSchema()


root
 |-- Frequency: integer (nullable = true)
 |-- AngleOfAttack: double (nullable = true)
 |-- ChordLength: double (nullable = true)
 |-- FreeStreamVelocity: double (nullable = true)
 |-- SuctionSideDisplacement: double (nullable = true)
 |-- SoundLevelDecibels: double (nullable = true)



In [ ]:
# your code goes here

df = df.withColumnRenamed("SoundLevel","SoundLevelDecibels")
df.show()


+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|           115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|           121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|           115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|           131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|           131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|           122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|           129.116|


### Task 12 - Save the dataframe in parquet formant, name the file as "NASA_airfoil_noise_cleaned.parquet"


In [ ]:
# your code goes here
df = df.repartition(1)
df.write.mode("overwrite").parquet("NASA_airfoil_noise_cleaned.parquet")

In [ ]:
# your code goes here
!ls -l NASA_airfoil_noise_cleaned.parquet

total 15
-rw-r--r-- 1 jupyterlab resources     0 Sep 13 12:51 _SUCCESS
-rw-r--r-- 1 jupyterlab resources 14754 Sep 13 12:51 part-00000-48623428-560b-4223-804d-ab9b2e1eb83d-c000.snappy.parquet


#### Part 1 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


## Part - 2 Create a  Machine Learning Pipeline


### Task 1 - Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe


In [ ]:
#your code goes here

df =  spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")


### Task 2 - Print the total number of rows in the dataset


In [ ]:
#your code goes here

rowcount4 = df.count()
print(rowcount4)



1499


### Task 3 - Define the VectorAssembler pipeline stage


Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [ ]:
#your code goes here

assembler = VectorAssembler(inputCols=["Frequency", "AngleOfAttack", "ChordLength", "FreeStreamVelocity", "SuctionSideDisplacement"], outputCol="features")



### Task 4 - Define the StandardScaler pipeline stage


Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [ ]:
#your code goes here

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")


### Task 5 - Define the StandardScaler pipeline stage


Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"


In [ ]:
#your code goes here

lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")

### Task 6 - Build the pipeline


Build a pipeline using the above three stages


In [ ]:
#your code goes here

# Build the pipeline
# All the stages of the pipeline are mentioned in the order of execution.
pipeline = Pipeline(stages=[assembler, scaler, lr])


### Task 7 - Split the data


In [ ]:
# Split the data into training and testing sets with 70:30 split.
# set the value of seed to 42
# the above step is very important. DO NOT set the value of seed to any other value other than 42.

#your code goes here

(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)


### Task 8 - Fit the pipeline


In [ ]:
# Fit the pipeline using the training data
# your code goes here

pipelineModel = pipeline.fit(trainingData)


23/09/13 12:51:57 WARN util.Instrumentation: [fd9332d8] regParam is zero, which might cause numerical instability and overfitting.
23/09/13 12:51:57 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/09/13 12:51:57 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/09/13 12:51:57 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/09/13 12:51:57 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


#### Part 2 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


## Part 3 - Evaluate the Model


### Task 1 - Predict using the model


In [ ]:
# Make predictions on testing data
# your code goes here

predictions = pipelineModel.transform(testingData)


### Task 2 - Print the MSE


In [ ]:
#your code goes here

evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
print("MSE =", mse)


MSE = 21.69767261083989


### Task 3 - Print the MAE


In [ ]:
#your code goes here

evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("MAE =", mae)


MAE = 3.674707483470427


### Task 4 - Print the R-Squared(R2)


In [ ]:
#your code goes here

evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R Squared =", r2)


R Squared = 0.5382661885816811


#### Part 3 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  21.7
Mean Absolute Error =  3.67
R Squared =  0.54
Intercept =  132.23


## Part 4 - Persist the Model


### Task 1 - Save the model to the path "Final_Project"


In [ ]:
# Save the pipeline model as "Final_Project"
!mkdir model_storage

In [ ]:
# Save the pipeline model as "Final_Project"
pipelineModel.write().overwrite().save("./Final_Project/")

#The overwrite method is used to overwrite the model if it already exists,
#and the save method is used to specify the path where the model should be saved.


### Task 2 - Load the model from the path "Final_Project"


In [ ]:
# Load the pipeline model you have created in the previous step

loadedPipelineModel = pipelineModel.load("./Final_Project/")

### Task 3 - Make predictions using the loaded model on the testdata


In [ ]:
# Use the loaded pipeline model and make predictions using testingData

predictions = loadedPipelineModel.transform(testingData)

### Task 4 - Show the predictions


In [ ]:
#show top 5 rows from the predections dataframe. Display only the label column and predictions
#your code goes here
predictions.select("prediction").show(5)

+------------------+
|        prediction|
+------------------+
| 125.1422875397131|
|122.70503286089041|
|123.66788978786391|
|127.56113462661494|
|120.83603611988016|
+------------------+
only showing top 5 rows



#### Part 4 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -4.0307
Coefficient for AngleOfAttack is -2.1385
Coefficient for ChordLength is -3.1762
Coefficient for FreeStreamVelocity is 1.6308
Coefficient for SuctionSideDisplacement is -2.0552


### Stop Spark Session


In [ ]:
spark.stop()

## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
